In [1]:
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from keras_applications import resnext
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
labels = pd.read_csv('Data/train_label.csv')
labels['Type'].value_counts()
sample = pd.read_csv('Data/sample_submission.csv')
num_train_images = 2520  # we choose 3300 images for this assignment. It works for a machine having 8Gb Ram. You can adjust it if your Ram is different. 
split_point = 2240 # split the data into training data [0:3000] and val data [3000:]
print('num_train_images:', num_train_images)
types = sample.columns[1:]
print('Types:', types)
num_classes = len(types)
print('num_classes:', num_classes)

num_train_images: 2520
Types: Index(['distribute', 'ineq', 'integral', 'limit', 'matrix', 'series', 'sqrt'], dtype='object')
num_classes: 7


In [3]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_width = 224

def get_image(filename):
    ########################################################################
    # TODO: Your code here...
    ########################################################################
    original = load_img(filename, target_size=(224,224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return image_batch[0]

In [4]:
x_train = np.zeros((num_train_images, img_width, img_width, 3), dtype=np.uint8)
y_train = np.zeros((num_train_images, num_classes), dtype=np.uint8)

count = 0

for i in range(num_train_images):
    x_train[i] = get_image('Data/train/%s.png' % labels['id'][i])
    pos_arrays = (types == labels['Type'][i]).nonzero() # recall that types is the array of classes
    pos = pos_arrays[0][0]
    y_train[i][pos] = 1
    count += 1
    if(count % 1000 == 0): print(count, 'images loaded')

1000 images loaded
2000 images loaded


# Inception with nothing changed

In [6]:
from tensorflow.keras.applications import inception_v3
inception_x_train = inception_v3.preprocess_input(x_train.copy())
inception_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features = inception_model.predict(inception_x_train, batch_size=64, verbose=1)
print(inception_features.shape)

40/40 [==============================] - 98s 2s/step
(2520, 5, 5, 2048)


In [21]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [12]:
from tensorflow.keras.models import Model

model_i = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               1024500   
                                                                 
 batch_normalization_95 (Bat  (None, 500)              2000      
 chNormalization)                                                
                                                                 
 activation_95 (Activation)  (None, 500)               0   

In [13]:
model_i.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i.fit(inception_features[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)


Epoch 1/30
18/18 [==============================] - 1s 34ms/step - loss: 1.3024 - accuracy: 0.5482 - val_loss: 0.4914 - val_accuracy: 0.8536
Epoch 2/30
18/18 [==============================] - 0s 24ms/step - loss: 0.6688 - accuracy: 0.7683 - val_loss: 0.1785 - val_accuracy: 0.9643
Epoch 3/30
18/18 [==============================] - 0s 22ms/step - loss: 0.5328 - accuracy: 0.8138 - val_loss: 0.2078 - val_accuracy: 0.9429
Epoch 4/30
18/18 [==============================] - 0s 26ms/step - loss: 0.4450 - accuracy: 0.8554 - val_loss: 0.2223 - val_accuracy: 0.9393
Epoch 5/30
18/18 [==============================] - 0s 23ms/step - loss: 0.4039 - accuracy: 0.8638 - val_loss: 0.1862 - val_accuracy: 0.9500
Epoch 5: early stopping


In [15]:
model_i.save('model_i.h5') 

In [17]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model.predict(image_batch)
        predictions = model_i.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)


In [18]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)

In [19]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.725


# Inception with Activation = Sigmoid

In [20]:
inception_x_train_sig = inception_v3.preprocess_input(x_train.copy())
inception_model_sig = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_sig = inception_model_sig.predict(inception_x_train_sig, batch_size=64, verbose=1)
print(inception_features_sig.shape)

40/40 [==============================] - 105s 3s/step
(2520, 5, 5, 2048)


In [23]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("sigmoid")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [24]:
model_i_sig = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_sig.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_4   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_8 (Dropout)         (None, 2048)              0         
                                                                 
 dense_8 (Dense)             (None, 500)               1024500   
                                                                 
 batch_normalization_192 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_192 (Activation)  (None, 500)              0   

In [25]:
model_i_sig.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [26]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_sig.fit(inception_features_sig[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_sig[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 37ms/step - loss: 1.3383 - accuracy: 0.5045 - val_loss: 1.0109 - val_accuracy: 0.6536
Epoch 2/30
18/18 [==============================] - 0s 25ms/step - loss: 0.8113 - accuracy: 0.7129 - val_loss: 0.7366 - val_accuracy: 0.7500
Epoch 3/30
18/18 [==============================] - 0s 25ms/step - loss: 0.6703 - accuracy: 0.7790 - val_loss: 0.3748 - val_accuracy: 0.9286
Epoch 4/30
18/18 [==============================] - 0s 25ms/step - loss: 0.5890 - accuracy: 0.8049 - val_loss: 0.4157 - val_accuracy: 0.9000
Epoch 5/30
18/18 [==============================] - 0s 24ms/step - loss: 0.5538 - accuracy: 0.8112 - val_loss: 0.3194 - val_accuracy: 0.9357
Epoch 6/30
18/18 [==============================] - 0s 24ms/step - loss: 0.5074 - accuracy: 0.8272 - val_loss: 0.3156 - val_accuracy: 0.9393
Epoch 7/30
18/18 [==============================] - 0s 25ms/step - loss: 0.4757 - accuracy: 0.8402 - val_loss: 0.2320 - val_accuracy: 0.9500
Epoch 8/30
18

In [27]:
model_i_sig.save('model_i_sig.h5') 

In [28]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_sig.predict(image_batch)
        predictions = model_i_sig.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [29]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7928571428571428


# Inception with Activation = Tanh

In [30]:
inception_x_train_tanh = inception_v3.preprocess_input(x_train.copy())
inception_model_tanh = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_tanh = inception_model_tanh.predict(inception_x_train_tanh, batch_size=64, verbose=1)
print(inception_features_tanh.shape)

40/40 [==============================] - 106s 3s/step
(2520, 5, 5, 2048)


In [31]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("tanh")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [52]:
model_i_tanh = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_tanh.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_7   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_14 (Dropout)        (None, 2048)              0         
                                                                 
 dense_14 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_383 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_383 (Activation)  (None, 500)              0   

In [53]:
model_i_tanh.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [54]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_tanh.fit(inception_features_tanh[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_tanh[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 33ms/step - loss: 1.9322 - accuracy: 0.4705 - val_loss: 1.9651 - val_accuracy: 0.0000e+00
Epoch 2/30
18/18 [==============================] - 0s 24ms/step - loss: 1.9118 - accuracy: 0.6460 - val_loss: 1.9878 - val_accuracy: 0.0000e+00
Epoch 3/30
18/18 [==============================] - 0s 23ms/step - loss: 1.8960 - accuracy: 0.7031 - val_loss: 2.0109 - val_accuracy: 0.0000e+00
Epoch 4/30
18/18 [==============================] - 0s 25ms/step - loss: 1.8801 - accuracy: 0.7469 - val_loss: 2.0319 - val_accuracy: 0.0000e+00
Epoch 4: early stopping


In [55]:
model_i_tanh.save('model_i_tanh.h5') 

In [56]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_tanh.predict(image_batch)
        predictions = model_i_tanh.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [41]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.6607142857142857


# Inception with Activation = softmax

In [42]:
inception_x_train_soft = inception_v3.preprocess_input(x_train.copy())
inception_model_soft = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_soft = inception_model_soft.predict(inception_x_train_soft, batch_size=64, verbose=1)
print(inception_features_soft.shape)

40/40 [==============================] - 108s 3s/step
(2520, 5, 5, 2048)


In [57]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("softmax")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [58]:
model_i_soft = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_soft.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_8   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_16 (Dropout)        (None, 2048)              0         
                                                                 
 dense_16 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_384 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_384 (Activation)  (None, 500)              0   

In [59]:
model_i_soft.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [60]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_soft.fit(inception_features_soft[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_soft[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 36ms/step - loss: 1.9327 - accuracy: 0.4665 - val_loss: 1.9618 - val_accuracy: 0.0000e+00
Epoch 2/30
18/18 [==============================] - 1s 30ms/step - loss: 1.9125 - accuracy: 0.6313 - val_loss: 1.9836 - val_accuracy: 0.0000e+00
Epoch 3/30
18/18 [==============================] - 0s 27ms/step - loss: 1.8964 - accuracy: 0.6897 - val_loss: 2.0077 - val_accuracy: 0.0000e+00
Epoch 4/30
18/18 [==============================] - 0s 27ms/step - loss: 1.8799 - accuracy: 0.7312 - val_loss: 2.0306 - val_accuracy: 0.0000e+00
Epoch 4: early stopping


In [61]:
model_i_soft.save('model_i_soft.h5') 

In [62]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_soft.predict(image_batch)
        predictions = model_i_soft.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [63]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.4857142857142857


# Inception with Activation = elu

In [64]:
inception_x_train_elu = inception_v3.preprocess_input(x_train.copy())
inception_model_elu = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_elu = inception_model_elu.predict(inception_x_train_elu, batch_size=64, verbose=1)
print(inception_features_elu.shape)

40/40 [==============================] - 105s 3s/step
(2520, 5, 5, 2048)


In [65]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("elu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [66]:
model_i_elu = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_elu.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_9   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_18 (Dropout)        (None, 2048)              0         
                                                                 
 dense_18 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_479 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_479 (Activation)  (None, 500)              0   

In [67]:
model_i_elu.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [68]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_elu.fit(inception_features_elu[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_elu[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 36ms/step - loss: 1.3425 - accuracy: 0.5772 - val_loss: 0.2682 - val_accuracy: 0.9179
Epoch 2/30
18/18 [==============================] - 0s 26ms/step - loss: 0.6230 - accuracy: 0.7835 - val_loss: 0.2078 - val_accuracy: 0.9214
Epoch 3/30
18/18 [==============================] - 0s 27ms/step - loss: 0.5189 - accuracy: 0.8313 - val_loss: 0.0956 - val_accuracy: 0.9536
Epoch 4/30
18/18 [==============================] - 0s 26ms/step - loss: 0.4300 - accuracy: 0.8509 - val_loss: 0.1491 - val_accuracy: 0.9393
Epoch 5/30
18/18 [==============================] - 1s 29ms/step - loss: 0.4037 - accuracy: 0.8661 - val_loss: 0.0655 - val_accuracy: 0.9714
Epoch 6/30
18/18 [==============================] - 1s 29ms/step - loss: 0.3750 - accuracy: 0.8638 - val_loss: 0.0416 - val_accuracy: 0.9964
Epoch 7/30
18/18 [==============================] - 0s 28ms/step - loss: 0.3468 - accuracy: 0.8737 - val_loss: 0.0739 - val_accuracy: 0.9643
Epoch 8/30
18

In [69]:
model_i_elu.save('model_i_elu.h5') 

In [70]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_elu.predict(image_batch)
        predictions = model_i_elu.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [71]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7642857142857142


# Inception with Activation = exponential

In [72]:
inception_x_train_ex = inception_v3.preprocess_input(x_train.copy())
inception_model_ex = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_ex = inception_model_ex.predict(inception_x_train_ex, batch_size=64, verbose=1)
print(inception_features_ex.shape)

40/40 [==============================] - 109s 3s/step
(2520, 5, 5, 2048)


In [73]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("exponential")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [74]:
model_i_expo = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_expo.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_10  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_20 (Dropout)        (None, 2048)              0         
                                                                 
 dense_20 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_574 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_574 (Activation)  (None, 500)              0  

In [75]:
model_i_expo.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [76]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_expo.fit(inception_features_ex[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_ex[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 39ms/step - loss: 3.7565 - accuracy: 0.4612 - val_loss: 4.0204 - val_accuracy: 0.1893
Epoch 2/30
18/18 [==============================] - 0s 26ms/step - loss: 1.8887 - accuracy: 0.6656 - val_loss: 0.9243 - val_accuracy: 0.8143
Epoch 3/30
18/18 [==============================] - 1s 29ms/step - loss: 1.2839 - accuracy: 0.7259 - val_loss: 0.5954 - val_accuracy: 0.8607
Epoch 4/30
18/18 [==============================] - 1s 29ms/step - loss: 1.1232 - accuracy: 0.7518 - val_loss: 0.5059 - val_accuracy: 0.8536
Epoch 5/30
18/18 [==============================] - 1s 27ms/step - loss: 0.9127 - accuracy: 0.7911 - val_loss: 0.1901 - val_accuracy: 0.9500
Epoch 6/30
18/18 [==============================] - 0s 26ms/step - loss: 0.7887 - accuracy: 0.8112 - val_loss: 0.2365 - val_accuracy: 0.9571
Epoch 7/30
18/18 [==============================] - 1s 33ms/step - loss: 0.7725 - accuracy: 0.7996 - val_loss: 0.2874 - val_accuracy: 0.9107
Epoch 8/30
18

In [77]:
model_i_expo.save('model_i_expo.h5') 

In [78]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_ex.predict(image_batch)
        predictions = model_i_expo.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [79]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7214285714285714


# Inception with Optimizer = SGD

In [80]:
inception_x_train_sgd = inception_v3.preprocess_input(x_train.copy())
inception_model_sgd = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_sgd = inception_model_sgd.predict(inception_x_train_sgd, batch_size=64, verbose=1)
print(inception_features_sgd.shape)

40/40 [==============================] - 99s 2s/step
(2520, 5, 5, 2048)


In [81]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [82]:
model_i_sgd = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_sgd.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_11  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_22 (Dropout)        (None, 2048)              0         
                                                                 
 dense_22 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_669 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_669 (Activation)  (None, 500)              0  

In [83]:
model_i_sgd.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])

In [84]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_sgd.fit(inception_features_sgd[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_sgd[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 33ms/step - loss: 2.2732 - accuracy: 0.2116 - val_loss: 1.7501 - val_accuracy: 0.2250
Epoch 2/30
18/18 [==============================] - 0s 24ms/step - loss: 1.7745 - accuracy: 0.3549 - val_loss: 1.7350 - val_accuracy: 0.2214
Epoch 3/30
18/18 [==============================] - 0s 24ms/step - loss: 1.5624 - accuracy: 0.4317 - val_loss: 1.6240 - val_accuracy: 0.2786
Epoch 4/30
18/18 [==============================] - 0s 24ms/step - loss: 1.4236 - accuracy: 0.4674 - val_loss: 1.4750 - val_accuracy: 0.4107
Epoch 5/30
18/18 [==============================] - 0s 27ms/step - loss: 1.3542 - accuracy: 0.5071 - val_loss: 1.2865 - val_accuracy: 0.5464
Epoch 6/30
18/18 [==============================] - 0s 24ms/step - loss: 1.2458 - accuracy: 0.5571 - val_loss: 1.1512 - val_accuracy: 0.6393
Epoch 7/30
18/18 [==============================] - 0s 23ms/step - loss: 1.2326 - accuracy: 0.5522 - val_loss: 1.0502 - val_accuracy: 0.7214
Epoch 8/30
18

In [85]:
model_i_sgd.save('model_i_sgd.h5') 

In [86]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_sgd.predict(image_batch)
        predictions = model_i_sgd.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [87]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7428571428571429


# Inception with Optimizer = RMS

In [88]:
inception_x_train_rms = inception_v3.preprocess_input(x_train.copy())
inception_model_rms = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_rms = inception_model_rms.predict(inception_x_train_rms, batch_size=64, verbose=1)
print(inception_features_rms.shape)

40/40 [==============================] - 109s 3s/step
(2520, 5, 5, 2048)


In [89]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [90]:
model_i_rms = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_rms.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_12  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_24 (Dropout)        (None, 2048)              0         
                                                                 
 dense_24 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_764 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_764 (Activation)  (None, 500)              0  

In [91]:
model_i_rms.compile(loss='categorical_crossentropy', optimizer="RMSprop", metrics=['accuracy'])

In [92]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_rms.fit(inception_features_rms[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_rms[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 46ms/step - loss: 1.0969 - accuracy: 0.6192 - val_loss: 0.3038 - val_accuracy: 0.9321
Epoch 2/30
18/18 [==============================] - 1s 33ms/step - loss: 0.6312 - accuracy: 0.7768 - val_loss: 0.2645 - val_accuracy: 0.9286
Epoch 3/30
18/18 [==============================] - 1s 33ms/step - loss: 0.5242 - accuracy: 0.8214 - val_loss: 0.1817 - val_accuracy: 0.9464
Epoch 4/30
18/18 [==============================] - 1s 33ms/step - loss: 0.4775 - accuracy: 0.8362 - val_loss: 0.1438 - val_accuracy: 0.9536
Epoch 5/30
18/18 [==============================] - 1s 33ms/step - loss: 0.4178 - accuracy: 0.8589 - val_loss: 0.1567 - val_accuracy: 0.9357
Epoch 6/30
18/18 [==============================] - 1s 34ms/step - loss: 0.3626 - accuracy: 0.8732 - val_loss: 0.0617 - val_accuracy: 0.9821
Epoch 7/30
18/18 [==============================] - 1s 33ms/step - loss: 0.3646 - accuracy: 0.8732 - val_loss: 0.0723 - val_accuracy: 0.9714
Epoch 8/30
18

In [93]:
model_i_rms.save('model_i_rms.h5') 

In [94]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_rms.predict(image_batch)
        predictions = model_i_rms.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [95]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.775


# Inception with Otpimizer = Adadlta

In [96]:
inception_x_train_ada = inception_v3.preprocess_input(x_train.copy())
inception_model_ada = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_ada = inception_model_ada.predict(inception_x_train_ada, batch_size=64, verbose=1)
print(inception_features_ada.shape)

40/40 [==============================] - 117s 3s/step
(2520, 5, 5, 2048)


In [97]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [98]:
model_i_ada = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_ada.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_13  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_26 (Dropout)        (None, 2048)              0         
                                                                 
 dense_26 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_859 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_859 (Activation)  (None, 500)              0  

In [99]:
model_i_ada.compile(loss='categorical_crossentropy', optimizer="Adadelta", metrics=['accuracy'])

In [100]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_ada.fit(inception_features_ada[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_ada[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 42ms/step - loss: 2.7064 - accuracy: 0.1326 - val_loss: 2.2608 - val_accuracy: 0.0036
Epoch 2/30
18/18 [==============================] - 1s 31ms/step - loss: 2.6408 - accuracy: 0.1353 - val_loss: 2.2451 - val_accuracy: 0.0036
Epoch 3/30
18/18 [==============================] - 1s 30ms/step - loss: 2.6439 - accuracy: 0.1357 - val_loss: 2.2369 - val_accuracy: 0.0036
Epoch 4/30
18/18 [==============================] - 0s 28ms/step - loss: 2.6344 - accuracy: 0.1357 - val_loss: 2.2279 - val_accuracy: 0.0036
Epoch 5/30
18/18 [==============================] - 0s 25ms/step - loss: 2.5930 - accuracy: 0.1496 - val_loss: 2.2190 - val_accuracy: 0.0179
Epoch 6/30
18/18 [==============================] - 1s 31ms/step - loss: 2.6186 - accuracy: 0.1446 - val_loss: 2.2123 - val_accuracy: 0.0179
Epoch 7/30
18/18 [==============================] - 1s 32ms/step - loss: 2.5949 - accuracy: 0.1344 - val_loss: 2.2004 - val_accuracy: 0.0214
Epoch 8/30
18

In [101]:
model_i_ada.save('model_i_ada.h5') 

In [102]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_ada.predict(image_batch)
        predictions = model_i_ada.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [103]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.18928571428571428


# Inception with Loss = MeanSquaredError

In [104]:
inception_x_train_mse = inception_v3.preprocess_input(x_train.copy())
inception_model_mse = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_mse = inception_model_mse.predict(inception_x_train_mse, batch_size=64, verbose=1)
print(inception_features_mse.shape)


40/40 [==============================] - 101s 2s/step
(2520, 5, 5, 2048)


In [105]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [106]:
model_i_mse = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_mse.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_14  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_28 (Dropout)        (None, 2048)              0         
                                                                 
 dense_28 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_954 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_954 (Activation)  (None, 500)              0  

In [108]:
model_i_mse.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])

In [109]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_mse.fit(inception_features_mse[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_mse[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 37ms/step - loss: 0.0940 - accuracy: 0.4960 - val_loss: 0.0846 - val_accuracy: 0.6250
Epoch 2/30
18/18 [==============================] - 0s 28ms/step - loss: 0.0518 - accuracy: 0.7464 - val_loss: 0.0262 - val_accuracy: 0.8571
Epoch 3/30
18/18 [==============================] - 1s 29ms/step - loss: 0.0434 - accuracy: 0.7817 - val_loss: 0.0192 - val_accuracy: 0.9000
Epoch 4/30
18/18 [==============================] - 1s 30ms/step - loss: 0.0342 - accuracy: 0.8317 - val_loss: 0.0283 - val_accuracy: 0.8643
Epoch 5/30
18/18 [==============================] - 0s 27ms/step - loss: 0.0312 - accuracy: 0.8536 - val_loss: 0.0216 - val_accuracy: 0.9000
Epoch 6/30
18/18 [==============================] - 0s 26ms/step - loss: 0.0305 - accuracy: 0.8545 - val_loss: 0.0142 - val_accuracy: 0.9250
Epoch 7/30
18/18 [==============================] - 0s 27ms/step - loss: 0.0273 - accuracy: 0.8665 - val_loss: 0.0100 - val_accuracy: 0.9571
Epoch 8/30
18

In [110]:
model_i_mse.save('model_i_mse.h5') 

In [111]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_mse.predict(image_batch)
        predictions = model_i_mse.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [121]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7928571428571428


# Inception with Loss = MeanAbsoluteError

In [122]:
inception_x_train_mae = inception_v3.preprocess_input(x_train.copy())
inception_model_mae = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_mae = inception_model_mae.predict(inception_x_train_mae, batch_size=64, verbose=1)
print(inception_features_mae.shape)

40/40 [==============================] - 107s 3s/step
(2520, 5, 5, 2048)


In [123]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [124]:
model_i_mae = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_mae.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_17  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_34 (Dropout)        (None, 2048)              0         
                                                                 
 dense_34 (Dense)            (None, 500)               1024500   
                                                                 
 batch_normalization_1051 (B  (None, 500)              2000      
 atchNormalization)                                              
                                                                 
 activation_1051 (Activation  (None, 500)              0  

In [125]:
model_i_mae.compile(loss='mean_absolute_error', optimizer="adam", metrics=['accuracy'])

In [126]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_mae.fit(inception_features_mae[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_mae[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 35ms/step - loss: 0.1587 - accuracy: 0.4790 - val_loss: 0.2521 - val_accuracy: 0.0536
Epoch 2/30
18/18 [==============================] - 0s 26ms/step - loss: 0.0911 - accuracy: 0.7237 - val_loss: 0.2453 - val_accuracy: 0.0893
Epoch 3/30
18/18 [==============================] - 0s 26ms/step - loss: 0.0714 - accuracy: 0.7826 - val_loss: 0.1228 - val_accuracy: 0.6179
Epoch 4/30
18/18 [==============================] - 1s 30ms/step - loss: 0.0613 - accuracy: 0.8156 - val_loss: 0.0731 - val_accuracy: 0.8000
Epoch 5/30
18/18 [==============================] - 0s 26ms/step - loss: 0.0539 - accuracy: 0.8455 - val_loss: 0.0425 - val_accuracy: 0.8536
Epoch 6/30
18/18 [==============================] - 0s 28ms/step - loss: 0.0481 - accuracy: 0.8562 - val_loss: 0.0367 - val_accuracy: 0.8964
Epoch 7/30
18/18 [==============================] - 0s 26ms/step - loss: 0.0444 - accuracy: 0.8714 - val_loss: 0.0333 - val_accuracy: 0.8964
Epoch 8/30
18

In [127]:
model_i_mae.save('model_i_mae.h5') 

In [ ]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_mae.predict(image_batch)
        predictions = model_i_mae.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [98]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.5392857142857143


# Inception with Loss = Hinge

In [6]:
from tensorflow.keras.applications import inception_v3
inception_x_train_hinge = inception_v3.preprocess_input(x_train.copy())
inception_model_hinge = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_hinge = inception_model_hinge.predict(inception_x_train_hinge, batch_size=64, verbose=1)
print(inception_features_hinge.shape)

40/40 [==============================] - 110s 3s/step
(2520, 5, 5, 2048)


In [8]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [9]:
from tensorflow.keras.models import Model

model_i_h = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_h.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 500)               1024500   
                                                                 
 batch_normalization_94 (Bat  (None, 500)              2000      
 chNormalization)                                                
                                                                 
 activation_94 (Activation)  (None, 500)               0     

In [10]:
model_i_h.compile(loss='hinge', optimizer="adam", metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_h.fit(inception_features_hinge[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_hinge[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 39ms/step - loss: 1.0087 - accuracy: 0.5116 - val_loss: 1.1391 - val_accuracy: 0.0000e+00
Epoch 2/30
18/18 [==============================] - 0s 26ms/step - loss: 0.9504 - accuracy: 0.7058 - val_loss: 1.1352 - val_accuracy: 0.0000e+00
Epoch 3/30
18/18 [==============================] - 0s 26ms/step - loss: 0.9310 - accuracy: 0.7696 - val_loss: 1.1060 - val_accuracy: 0.1143
Epoch 4/30
18/18 [==============================] - 1s 34ms/step - loss: 0.9199 - accuracy: 0.8112 - val_loss: 0.9598 - val_accuracy: 0.7036
Epoch 5/30
18/18 [==============================] - 0s 28ms/step - loss: 0.9138 - accuracy: 0.8321 - val_loss: 0.9238 - val_accuracy: 0.8036
Epoch 6/30
18/18 [==============================] - 1s 32ms/step - loss: 0.9064 - accuracy: 0.8576 - val_loss: 0.9001 - val_accuracy: 0.8857
Epoch 7/30
18/18 [==============================] - 1s 28ms/step - loss: 0.9042 - accuracy: 0.8625 - val_loss: 0.8960 - val_accuracy: 0.8857
Epoch

In [13]:
model_i_h.save('model_i_h.h5') 

In [14]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_hinge.predict(image_batch)
        predictions = model_i_h.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [15]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7892857142857143


# Inception with Loss = SquaredHinge

In [16]:
inception_x_train_sh = inception_v3.preprocess_input(x_train.copy())
inception_model_sh = inception_v3.InceptionV3(weights='imagenet', include_top=False)
inception_features_sh = inception_model_sh.predict(inception_x_train_sh, batch_size=64, verbose=1)
print(inception_features_sh.shape)

40/40 [==============================] - 102s 3s/step
(2520, 5, 5, 2048)


In [17]:
inputs = Input(shape = (5, 5, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [18]:
model_i_sh = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_i_sh.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 5, 5, 2048)]      0         
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               1024500   
                                                                 
 batch_normalization_189 (Ba  (None, 500)              2000      
 tchNormalization)                                               
                                                                 
 activation_189 (Activation)  (None, 500)              0   

In [20]:
model_i_sh.compile(loss='squared_hinge', optimizer="adam", metrics=['accuracy'])

In [21]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_i_sh.fit(inception_features_sh[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(inception_features_sh[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 1s 38ms/step - loss: 1.1019 - accuracy: 0.5268 - val_loss: 1.1163 - val_accuracy: 0.5286
Epoch 2/30
18/18 [==============================] - 0s 27ms/step - loss: 0.9966 - accuracy: 0.7504 - val_loss: 0.9947 - val_accuracy: 0.7821
Epoch 3/30
18/18 [==============================] - 0s 28ms/step - loss: 0.9691 - accuracy: 0.8054 - val_loss: 0.9296 - val_accuracy: 0.8786
Epoch 4/30
18/18 [==============================] - 0s 27ms/step - loss: 0.9526 - accuracy: 0.8388 - val_loss: 0.9047 - val_accuracy: 0.9286
Epoch 5/30
18/18 [==============================] - 0s 27ms/step - loss: 0.9390 - accuracy: 0.8594 - val_loss: 0.9013 - val_accuracy: 0.9250
Epoch 6/30
18/18 [==============================] - 0s 26ms/step - loss: 0.9367 - accuracy: 0.8665 - val_loss: 0.8933 - val_accuracy: 0.9357
Epoch 7/30
18/18 [==============================] - 0s 26ms/step - loss: 0.9284 - accuracy: 0.8826 - val_loss: 0.8929 - val_accuracy: 0.9357
Epoch 8/30
18

In [22]:
model_i_sh.save('model_i_sh.h5') 

In [23]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = inception_v3.preprocess_input(image_batch)
        feature_input = inception_model_sh.predict(image_batch)
        predictions = model_i_sh.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [24]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7857142857142857
